In [2]:
# based on OSSI_04_Final_scripts from Ruben
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta


import xarray as xr
import matplotlib.pyplot as plt
import sys
sys.path.append(r'C:\Users\ruro\OneDrive - Boskalis\Documents\python\Hybrid_Dune\Pressure_sensors\S1\OSSI_04')
from scipy.signal import welch

#import puv 



In [3]:
def ossi_data_reader(datafolder):
    '''
    author: Paul van Wiechen

    Function to read all WLOG_XXX files in a certain subfolder.
    Make sure that only WLOG_XXX files are in this folder and no other files.
    Only WLOG_XXX files with minimally 2 rows are appended to the dataframe.
    A correct WLOG_XXX file should contain a first line with OSSI configuration, and a second line (third row) with starting time
    Timestep and sampling frequency are retrieved from the first row. Starting time from the next row
    Returns a dataframe with a time column and pressure column in dbars
    '''

    ossi = pd.DataFrame({
        't': [],
        'p': []})

    directory = str(datafolder)

    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)

        # checking if it is a file
        if os.path.isfile(f):
            print('Currently concatenating file ' + f)
            ossi_raw = pd.read_csv(f, header=None, nrows=4, sep=',')
            if len(ossi_raw.index) > 2:
                t_0 = datetime(int(str(20) + ossi_raw[0][1][1:]), int(ossi_raw[1][1][1:]), int(ossi_raw[2][1][1:]),
                               int(ossi_raw[3][1][1:]), int(ossi_raw[4][1][1:]), int(ossi_raw[5][1][1:]))
                dt = 1 / float(ossi_raw[6][0][1:])
                ossi_tot = pd.read_csv(f, skiprows=3, usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], header=None,
                                       sep=',', skipinitialspace=True).to_numpy().flatten()
                ossi_temp = pd.DataFrame({
                    't': np.array([t_0 + timedelta(seconds=dt * i) for i in range(len(ossi_tot))]),
                    'p': ossi_tot})

                ossi_temp.dropna(inplace=True)
                ossi_temp['p'] = ossi_temp['p'] * 1e5  # Bar to Pa

                ossi = pd.concat([ossi, ossi_temp], ignore_index=True)

    ossi['p'] = pd.to_numeric(ossi['p'])
    ossi['t'] = pd.to_datetime(ossi['t'])

    return ossi.set_index('t')


In [12]:
experimentFolder = r"O:\HybridDune experiment\data RBR, OSSI\Ossi data"    #Grand map
subfolder_in_all = ['OSSI 1', 'OSSI 2', 'OSSI 3', 'OSSI 4', 'OSSI 5', 'OSSI 6', 'OSSI 7', 'OSSI 8'] # subfolder where file is sitting within experimentFolder (on O drive Daan)
for i in range(1,8):
    subfolder_in = subfolder_in_all[i]
    dfp = ossi_data_reader(os.path.join(experimentFolder,subfolder_in))                                                #specific map for data OSSI

    instrument = subfolder_in#'OSSI_04'          # Specific name instrument
    sf = 20                          # Hz, sampling frequency
    xRD = 72478.479                   # location x-coord
    yRD = 452138.636                   # location y-coord
    serial_number = '18.09.00.08'    # unique serial number #not correct

    rho = 1027      # Density of sea water
    g = 9.8125        # Gravitational force
    zb = 0.687      # Height of sea bed NAP   #23/12/2024
    zi = 1.339      # Height of instrument NAP

    ds = dfp.to_xarray()
    ds['p'] = ds.p.astype('int32')
    ds.p.attrs = {'long_name': 'pressure', 'units': 'Pa'}

    # add global attribute metadata
    ds.attrs = {
        'Conventions': 'CF-1.6',
        'name': '{}'.format(instrument),
        'instrument': '{}'.format(instrument),
        'instrument type': 'OSSI',
        'instrument serial number': '{}'.format(serial_number),
        'epsg': 28992,
        'x': xRD,
        'y': yRD,
        'sf': 20,
        'time zone': 'UTC+2',
        'summary': 'Hybrid-dune experiment',
        'contact person': 'Ruben Rosman',
        'emailadres': 'r.g.c.rosman@student.tudelft.nl',
        'construction datetime': datetime.now().strftime("%d-%b-%Y (%H:%M:%S)"),
        'version': 'v1',
        'version comments': 'constructed with xarray'}

    #if nothing else, at least specify lossless zlib compression
    encoding = {var: {"zlib": True, "complevel": 4} for var in list(ds.data_vars) + list(ds.coords)}  # Apply deflate compression to all variables and coordinates in netCDF

    # save to file
    if not os.path.isdir(os.path.join(experimentFolder,'raw_netcdf')):
        os.mkdir(os.path.join(experimentFolder,'raw_netcdf'))
    ncFilePath = os.path.join(experimentFolder, 'raw_netcdf', instrument + '.nc')
    ds.to_netcdf(ncFilePath, encoding=encoding)


Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 2\WLOG_001.CSV


C:\Users\dpoppema\AppData\Local\Temp\ipykernel_23452\1382462514.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ossi = pd.concat([ossi, ossi_temp], ignore_index=True)


Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 2\WLOG_002.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 2\WLOG_003.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 2\WLOG_004.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 2\WLOG_005.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 2\WLOG_006.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 2\WLOG_007.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 2\WLOG_008.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 2\WLOG_009.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 2\WLOG_010.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 2\WLOG_011.CSV
Currently 

C:\Users\dpoppema\AppData\Local\Temp\ipykernel_23452\1382462514.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ossi = pd.concat([ossi, ossi_temp], ignore_index=True)


Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 3\WLOG_005.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 3\WLOG_006.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 3\WLOG_007.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 3\WLOG_008.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 3\WLOG_009.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 3\WLOG_010.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 3\WLOG_011.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 3\WLOG_012.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 3\WLOG_013.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 3\WLOG_014.CSV
Currently 

...

Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 6\WLOG_003.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 6\WLOG_004.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 6\WLOG_005.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 6\WLOG_006.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 6\WLOG_007.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 6\WLOG_008.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 6\WLOG_009.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 6\WLOG_010.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 6\WLOG_011.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 6\WLOG_012.CSV
Currently 

C:\Users\dpoppema\AppData\Local\Temp\ipykernel_23452\1382462514.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ossi = pd.concat([ossi, ossi_temp], ignore_index=True)


Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 7\WLOG_003.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 7\WLOG_004.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 7\WLOG_005.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 7\WLOG_006.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 7\WLOG_007.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 7\WLOG_008.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 7\WLOG_009.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 7\WLOG_010.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 7\WLOG_011.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 7\WLOG_012.CSV
Currently 

C:\Users\dpoppema\AppData\Local\Temp\ipykernel_23452\1382462514.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ossi = pd.concat([ossi, ossi_temp], ignore_index=True)


Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 8\WLOG_003.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 8\WLOG_004.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 8\WLOG_005.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 8\WLOG_006.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 8\wlog_007.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 8\WLOG_008.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 8\WLOG_009.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 8\WLOG_010.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 8\WLOG_011.CSV
Currently concatenating file O:\HybridDune experiment\data RBR, OSSI\Ossi data\OSSI 8\WLOG_012.CSV
Currently 

In [11]:
instrument = subfolder_in#'OSSI_04'          # Specific name instrument
ncFilePath = os.path.join(experimentFolder, 'raw_netcdf', instrument + '.nc')
ds.to_netcdf(ncFilePath, encoding=encoding)